In [1]:
import numpy as np
import pandas as pd
import datetime
import time
import requests
import json
from geopy import distance

In [2]:
def clean(df):
    df01 = df[df["Trip  Duration"] != 0]
    df01 = df01[df01["End Station Id"].notnull()]
    df01 = df01[df01["Bike Id"].notnull()]
    return df01

In [3]:
def conversion(df):
    df['Start Time'] = pd.to_datetime(df['Start Time'])
    df['End Time'] = pd.to_datetime(df['End Time'])
    df['Start Station Id'] = df['Start Station Id'].astype("string")
    df['End Station Id'] = df['End Station Id'].astype("int")
    df['End Station Id'] = df['End Station Id'].astype("string")
    df['Bike Id'] = df['Bike Id'].astype("int")
    df['Bike Id'] = df['Bike Id'].astype("string")
    return df

In [4]:
def calculation(df):
    df["routes"] = df["Start Station Id"].str.cat("-" + df["End Station Id"])
    
    routes = df[['Start Station Id', 'End Station Id', "routes"]].drop_duplicates()
    routes = routes.merge(bikeshare_stations, how='inner', left_on='Start Station Id', right_on='station_id') \
       .merge(bikeshare_stations, how='inner', left_on='End Station Id', right_on='station_id', suffixes=['_start', '_end']) \
       .drop_duplicates()
    routes['distance'] = routes.apply(lambda x: distance.distance((x['lat_start'], x['lon_start']), (x['lat_end'], x['lon_end'])).m, axis=1)
    
    df01 = df.merge(routes[['routes', 'distance']], how='left', on='routes')
    df01["avg_speed"] = df01["distance"]/ (df01["Trip  Duration"])
    df01 = df01[(df01["avg_speed"] < 30) & (df01["Trip  Duration"] != 2)]
    df01 = df01[df01["Trip  Duration"] < 43200]
    return df01